In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))
train=pd.read_csv('../input/labeledTrainData.tsv',delimiter='\t',quoting=3)
# Any results you write to the current directory are saved as output.

['unlabeledTrainData.tsv', 'labeledTrainData.tsv', 'testData.tsv', 'sampleSubmission.csv']


In [9]:
train.head(4)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."


In [10]:
train.shape

(25000, 3)

In [11]:
train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [12]:
train["review"][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

**Limpeza de dados e Processamento de texto**

O dataset é resultado de DataMining, portanto deixa "impurezas" que podem atraplhar nosso modelo, neste caso temos presença de tags de HTML e usando o pacote beautiful soup conseguimos realizar a limpeza dessas tags nos dados.

In [13]:
from bs4 import BeautifulSoup
example=BeautifulSoup(train['review'][0], "lxml" )
print (train['review'][0])
print (example.get_text())

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

Conseiderando como limpar o texto, temos que pensar sobre os problemas que surgirão conforme a estrutura dos dados presente. Um que faz sentido é remover a pontuação, um spoiler é que não precisamos dessa estrutura de texto quando aplicamos Bag of Words, pois a posição da palavra na frase será irrelevânte ao modelo que iremos aplicar. De outra forma, estamos tentanto solucionar um problema de analise de sentimentos, Positivos e Negativos, portanto as pontuações podem expressar sentimentos e também possivelmente ser importante, como por exemplo  "!!!" ou ":-("  que podem carregar uma boa informação para auxiliar na predição, e portanto temos que talvez trata-los como "palavras" . No tutorial, pela simplicidade, nós removemos todas a pontuações, mas é algo que pode ser melhorado.

Também removeremos números, mas se tiver alguma funcionalidade no futuro pode fazer sentido usa-los, então vamos apenas repor por um valor nulo.

Para remover pontuação e números, vamos usar um famoso pacote de ferramentas que lidam com expressões relgulares, chamado **re**

In [14]:
import re
letters_only=re.sub("[^a-zA-Z]"," ",example.get_text())
print(letters_only)

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

Aplicamos nas avaliações em que todas as letras fiquem na forma minuscúla, para ter uniciade nas escrita das palavras. Também vamos realizar a **tokenização**, onde o texto da avaliação é transformado um array, cada espaço do array conresponde à uma palavra desse mesmo texto. A técnica **tokenização** de é chave para realizar Bag of Words e realizar NLP em modelos de machine learning. 

In [15]:
lower_case=letters_only.lower()
words=lower_case.split()

Com a **tokenização** vamos lidar com a frequencia da ocorrencia das palavras em cada avaliação do dataset, não com o significado das palavras. 

Temos palavras que são muito comuns em textos, mas não represetam um grandes significados na semântica, que geralmente são artigos, pronomes e preposições, então chamamos essas palavras de **"stop words"**, que no inglês temos palavras de exemplo:  **"a"**, **"and"**,** "is"**, e **"the"**. Por coviniência temos pacotes no Python que possuem uma lista de palavras consideradas como **"stop words"** e também realizam esse filtro, para realizar esse processamento. temos o pacote  **Python Natural Language Toolkit (NLTK)** que também possue muito outras ferramentas para lidar com problemas de processamento de linguagem natual.


In [16]:
#import nltk
#nltk.download()

In [17]:
from nltk.corpus import stopwords
#lista de stop words da lingua Inglêsa
print (stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [18]:
#Filtro das palavras não stop words presentes na avaliação 
words=[w for w in words if not w in stopwords.words("english")]
print(words)

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 

In [19]:
def review_to_words(raw_review):
    review_text=BeautifulSoup(raw_review, "lxml").get_text()
    letters_only=re.sub('[^a-zA-Z]',' ',review_text)
    words=letters_only.lower().split()
    stops=set(stopwords.words('english'))
    meaningful_words=[w for w in words if not w in stops]
    return(' '.join(meaningful_words))

Aplicamos uma técnica que não escopo desse estudo, uso de outros tipos de estrutura de dados para processar os dados, neste caso estamos transformando os dados em **arrays** do tipo **numpy**, com foco em velocidade, pois vamos chamar funções looping em várias instâncias, loop no python não é necessariamente efeiciente e ainda mais manipulando estrutura de dados em forma de lsitas, portanto quanto mais rápido melhor. 

Então criamos uma funação **review_to_words** que irá aplicar todas essas transformações no dataset de uma vez.

In [20]:
clean_review=review_to_words(train['review'][0])
print(clean_review)

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

In [21]:
num_reviews=train['review'].size
num_reviews

25000

In [22]:
clean_train_review=[]
for i in range(0,num_reviews):
    clean_train_review.append(review_to_words(train['review'][i]))

Criando Features com a Técnica Bag of Words (scikit-learn)
Now that we have our training reviews tidied up, how do we convert them to some kind of numeric representation for machine learning? One common approach 
Com as avaliações formatadas, como converter elas em uma represemtação numérica para machine learning? Uma abordagem comum é a técnica Bag of Words, consiste no modelo usar o vocubulario de todos o documentos analisados (neste caso avaliações de filmes), literalmente as informações viram um "Saco de palavras", cada documento é levado em conta quais palavras do voculabulario foram utilizadas na formação deste mesmo documento. Segue exemplo em duas sentenças:

Frase 1: "The cat sat on the hat"

Frase 2: "The dog ate the cat and the hat"

Para as duas sentenças, o vocabulo segue:

{ the, cat, sat, on, hat, dog, ate, and }

Para conseguir o "saco de palavras", contamos o numero de vezes que a palavra ocorre em cada Frase. Na sentença 1, "the" aparece duas vezes, também as palavras "cat", "sat", "on", e "hat" cada aparece uma vez, então o novo registro da senteça 1 numéricamente fica:

{ the, cat, sat, on, hat, dog, ate, and }

Frase 1: { 2, 1, 1, 1, 1, 0, 0, 0 }

Frase 2: { 3, 1, 0, 0, 1, 1, 1, 1}

In the IMDB data, we have a very large number of reviews, which will give us a large vocabulary. To limit the size of the feature vectors, we should choose some
Na base de dados IMDB, temos um grande numero de avaliações, que nós dá um grande vocábulo. Para não hiperdimencionalizar o vetor de Features e causar riscos de performances no modelo, vamos escolher um espaço amostal  máximo de vocábulos. Abaixo, utilizaremos as 5000 palavras mais frequentes (lembrando que tiramos as stops words).

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(analyzer='word',tokenizer=None,preprocessor = None, stop_words = None,max_features = 5000)
train_data_features=vectorizer.fit_transform(clean_train_review)
train_data_features=train_data_features.toarray()

In [24]:
train_data_features.shape

(25000, 5000)

In [25]:
train_data_features[1]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [26]:
vcab=vectorizer.get_feature_names()
print(vcab)

['abandoned', 'abc', 'abilities', 'ability', 'able', 'abraham', 'absence', 'absent', 'absolute', 'absolutely', 'absurd', 'abuse', 'abusive', 'abysmal', 'academy', 'accent', 'accents', 'accept', 'acceptable', 'accepted', 'access', 'accident', 'accidentally', 'accompanied', 'accomplished', 'according', 'account', 'accuracy', 'accurate', 'accused', 'achieve', 'achieved', 'achievement', 'acid', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'activities', 'actor', 'actors', 'actress', 'actresses', 'acts', 'actual', 'actually', 'ad', 'adam', 'adams', 'adaptation', 'adaptations', 'adapted', 'add', 'added', 'adding', 'addition', 'adds', 'adequate', 'admire', 'admit', 'admittedly', 'adorable', 'adult', 'adults', 'advance', 'advanced', 'advantage', 'adventure', 'adventures', 'advertising', 'advice', 'advise', 'affair', 'affect', 'affected', 'afford', 'aforementioned', 'afraid', 'africa', 'african', 'afternoon', 'afterwards', 'age', 'aged', 'agent', 'agents', 'ages', 'aging', 'ago', 'ag

**Random Forest**

Neste ponto, temos um dataset totalmente formatado e com a aplicação da tokeninzação, BOW e transformação em array. Portanto vamos reliazar um aprendizado supervisionado usando esses dados, primeiramente vamos dividir o dataset em treino (X) e validação (Y) para poder avaliar esse modelo na predição quais avaliações futuro serão positivas ou negativas.

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data_features, train["sentiment"], test_size=0.2, random_state=42)

In [28]:
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(n_estimators=100, random_state=42)
%time forest = forest.fit( X_train, y_train )
result = forest.predict(X_test)

CPU times: user 1min 49s, sys: 876 ms, total: 1min 50s
Wall time: 1min 50s


In [29]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, result)

0.8446

In [30]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=42, solver='lbfgs')

In [31]:
clf = clf.fit( X_train, y_train )
result = clf.predict(X_test)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, result)

0.8588

In [33]:
word = "I really loved that movie, it was awesome!"

In [34]:
sample = review_to_words(word)
sample

'really loved movie awesome'

In [35]:
clean_train_review=[]
clean_train_review.append(review_to_words(word))
clean_train_review

['really loved movie awesome']

In [36]:
sample_final=vectorizer.transform(clean_train_review)
sample_final=sample_final.toarray()
sample_final

array([[0, 0, 0, ..., 0, 0, 0]])

In [37]:
result = forest.predict(sample_final)

In [38]:
result

array([1])

In [46]:
def predict_coment(word):
    to_array=[]
    to_array.append(review_to_words(word))
    sample_final=vectorizer.transform(to_array)
    sample_final=sample_final.toarray()
    result = clf.predict(sample_final)
    if  result[0] == 1:
        label = 'Positive'
    else:
        label = 'Negative'
        
    return label, word

In [47]:
predict_coment('I really hated that movie, it was so bad!')

('Negative', 'I really hated that movie, it was so bad!')

In [48]:
test = "I've liked Brie Larson in other films, but she showed ZERO range in this. When your main character in a superhero movie is unwatchable, you already have a problem. In addition, Captain Marvel has no weaknesses, which kills the tension immediately. There is no point at which you feel she is in any danger of losing, or any danger at all for that matter. It's an OK origin story, but it makes no sense as to WHY she's supposedly so powerful. The cat was good."

https://www.imdb.com/review/rw4704566/?ref_=tt_urv

In [49]:
predict_coment(test)

('Negative',
 "I've liked Brie Larson in other films, but she showed ZERO range in this. When your main character in a superhero movie is unwatchable, you already have a problem. In addition, Captain Marvel has no weaknesses, which kills the tension immediately. There is no point at which you feel she is in any danger of losing, or any danger at all for that matter. It's an OK origin story, but it makes no sense as to WHY she's supposedly so powerful. The cat was good.")

In [50]:
test2 = "We loved Green Book along with the sold-out crowd who applauded loudly at the end. Based on true story of piano virtuoso, Don Shirley's road trip through the south during the 60's, the film pays tribute to his genius and courage as a black man who tries hard to soar above the ugliness of the times. The elegant trappings of his home and his success as a concert pianist leave him arrogantly cold and lonely, but his life begins to change when he hires Tony as his road trip driver. With a history as nightclub bouncer with Mafia connections in New York, Tony is the antithesis of Don's perfection and their evolving relationship on the road makes the movie soar above the ordinary and become magical. Viggo and Ali in the main roles are remarkable, and it's funny and endearing to watch them discard stereotypes and discover their mutual humanity. What we liked best is the movie teaches without preaching, it all unfolds through a myriad of natural moments between two great actors and a strong supporting cast. Top all this off with a really good soundtrack, excellent direction and production values, and an audience that laughed out loud and vigorously applauded at the end...this one is a true 10!"

https://www.imdb.com/review/rw4477733/?ref_=tt_urv

In [51]:
predict_coment(test2)

('Positive',
 "We loved Green Book along with the sold-out crowd who applauded loudly at the end. Based on true story of piano virtuoso, Don Shirley's road trip through the south during the 60's, the film pays tribute to his genius and courage as a black man who tries hard to soar above the ugliness of the times. The elegant trappings of his home and his success as a concert pianist leave him arrogantly cold and lonely, but his life begins to change when he hires Tony as his road trip driver. With a history as nightclub bouncer with Mafia connections in New York, Tony is the antithesis of Don's perfection and their evolving relationship on the road makes the movie soar above the ordinary and become magical. Viggo and Ali in the main roles are remarkable, and it's funny and endearing to watch them discard stereotypes and discover their mutual humanity. What we liked best is the movie teaches without preaching, it all unfolds through a myriad of natural moments between two great actors a